In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers.models import Pooling

pooling_layer = Pooling(768)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "../../data/models/distilbert-base-finetuned-twitter15-tf/checkpoint-650/",
    output_hidden_states=True, num_labels=2)

model.to("cuda")
print("Loading done")

Loading done


In [2]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs.to("cuda")
    token_logits = model(inputs['input_ids'])

    embeddings = token_logits.hidden_states[-1]

    inputs = {
        'attention_mask': inputs['attention_mask'],
        'token_embeddings': embeddings,
        'cls_token_embeddings': None
    }
    result = pooling_layer.forward(inputs)
    result = result['sentence_embedding'].cpu().detach().numpy()

    return result

def encode_and_save(texts, savefile = None):
    if not savefile:
        print("Please define savefile")
        return
    
    vectors = []
    for text in texts:
        vectors.append(encode_text(text))
    
    vectors = np.array(vectors).reshape((len(texts), 768))
    
    with open(f"../../data/vectors/{savefile}.txt", "wb") as f:
        np.savetxt(f, vectors, delimiter=',', fmt='%s')

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/twitter15-tf_dataset.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,tvt2
0,551099691702956032,rip elly may clampett: so sad to learn #beverl...,True,training
1,514106273852174337,just in: missing afghan soldiers found trying ...,True,training
2,495366618818830336,#riphulkhogan my heart is ripping like your sh...,False,training
3,532206910796468224,a chick-fil-a manager allegedly banned this hi...,False,training
4,560187970389819392,islamic tribunal using sharia law in texas has...,False,validation


In [4]:
data.shape

(742, 4)

In [5]:
savefile = "Twitter15-TF_DistilBERT_base_finetuned_vectors"
texts = data['tweet_text'].tolist()

encode_and_save(texts, savefile)

In [6]:
vectors = np.loadtxt(f"../../data/vectors/{savefile}.txt", delimiter=",")
vectors.shape

(742, 768)